In [ ]:
from math import pi

import numpy as np
import numpy.linalg as npl

import scipy as sp
import scipy.linalg as spl
import scipy.sparse as sps
import scipy.sparse.linalg as sspl

import matplotlib.pyplot as plt

import holoviews as hv
import param
import panel as pn

hv.extension('bokeh')
import bokeh.plotting as bkplt

from tqdm import tqdm

In [ ]:
kappa = 1
s = 1
r_max = 1
sigma = 0.5
theta = 4.5
N = 100
a = 0
b = 10

In [ ]:
def A(dz, N):
    N = N - 2
    tmp = np.ones((N - 1))
    diag = -2 * np.ones(N)
    diag[0] /= 2
    diag[-1] /= 2
    return sps.diags((tmp, diag, tmp),(-1, 0, 1)) * sigma**2 / (dz**2)

def source(Z, mu=5, sigma=sigma):
    return (.5 / (2 * pi * sigma**2)**.5) * np.exp(-0.5 * ((Z - mu)**2) / sigma**2)

def get_r(Z,rmax,s,theta):
    return rmax - s * (Z - theta)**2

def get_rho(U, dz=None):
    if dz is None:
        dz = (b - a) / N
    
    return dz * ((U[..., 0] + U[..., -1]) / 2 + np.sum(U[..., 1:-1], axis=-1))
#     return dz*(U[0]/2 + U[-1]/2 + np.sum(U[1:-1]))

def solve_explicit(A, U, Z, rho, dz, dt, theta):
    r = get_r(Z, r_max, s, theta)
    tmp1 = sps.eye(A.shape[0])
    tmp2 = dt * A
    tmp3 = (dt * (r - kappa * rho))
    U_next = (tmp1 + tmp2).dot(U) + (U * tmp3)
    return U_next

def solve_implicit(A, U, Z, rho, dz, dt, theta):
    r = get_r(Z, r_max, s, theta)
    FU = F(U, r, rho)
    
    lhs = (np.eye(A.shape[0]) - dt * A)
    rhs = U + dt * FU
    return sspl.cg(lhs, rhs)[0]

def F(U, r, rho):
    return U * (r - kappa * rho)

def solve_splitting(A, U, Z, rho, dz, dt, theta):
    tempU1 = sspl.cg(np.eye(A.shape[0]) - dt / 2 * A, U)[0]
    
    r = get_r(Z, r_max, s, theta)
    
    tempU2 = tempU1 + dt * F(tempU1, r, rho)
    return sspl.cg(np.eye(A.shape[0]) - dt / 2 * A, tempU2)[0]

In [ ]:
def mean(Z, U):
    return np.sum(U * Z, axis=-1) / np.sum(U, axis=-1)

def var(Z, U):
    mu = mean(Z, U)
    return np.sum(U * Z**2, axis=-1) / np.sum(U, axis=-1) - mu**2 

In [ ]:
def solve(solver, dt, T, moving_theta=False, noise_theta=False, **theta_args):
    dz = (b - a) / N
    nbT = int(T / dt)
    
    A_mat = A(dz, N)
    
    Z = np.linspace(a, b, N)
    U = np.empty(N)
    
    # Initialisation of U_0
    U[1:-1] = np.copy(source(Z[1:-1]))
    U[0] = U[1]
    U[-1] = U[-2]
    
    # Arrays that saves the state of the system through time
    Us = np.empty((nbT + 1, N))
    RHO = np.empty(nbT + 1)
    
    THETA = np.ones(nbT + 1) * theta_args['theta']
    
    if moving_theta:
        MOVING = np.arange(nbT + 1) * theta_args['speed']
        THETA += MOVING
    
    if noise_theta:
        NOISE = np.random.normal(loc=0., scale=theta_args['std_noise'], size=nbT + 1)
        THETA += NOISE
    
    for i in tqdm(range(0, nbT + 1)):
        rho = get_rho(U, dz)
        
        Us[i] = np.copy(U)
        
        U[1:-1] = solver(A_mat, U[1:-1],  Z[1:-1], rho, dz, dt, THETA[i])
        U[0] = U[1]
        U[-1] = U[-2]
        
    return Z, np.array(Us), THETA

In [ ]:
def plot_2D(Us, dt, T, dynamic=True, nb_images=50):
    Z = np.linspace(a, b, N)
    time_step = Us.shape[0] // nb_images
    
    curve_dict = {round(dt * i, ndigits=1): hv.Curve((Z, Us[i])) for i in range(0, Us.shape[0], time_step)}
    
    if dynamic:
#         curve_dict = {i: hv.Curve((Z, Us[i])).opts(title=f'Approximated solution at t = {dt * i:.1f}') for i in range(0, Us.shape[0], time_step)}
        plot = hv.HoloMap(curve_dict, kdims='Time')
    
    else:
        plot = hv.NdOverlay(curve_dict, kdims='Time').opts(legend_position='right')
        
    return hv.output(plot.opts(width=600, toolbar=None))
    
def plot_params(Z, Us, THETA):
    RHO = get_rho(Us)
    MEAN = mean(Z, Us)
    VAR = var(Z, Us)
    
    options = {'shared_axes': False, 'toolbar': None}
        
    curves = []
    
    curves += [hv.NdOverlay({theta_name: hv.Curve(VALUE, 'Iteration', 'Theta').opts(alpha=alpha) for VALUE, theta_name, alpha in zip(
        [MEAN, THETA], ['Current', 'Optimal'], [1., .5]
    )}, kdims='Thetas').opts(title='Evolution of theta (mean)')]
    
    curves += [hv.Curve(VALUE, 'Iteration', val_name.capitalize()).opts(title=f'Evolution of {val_name}') for VALUE, val_name in zip(
        [VAR, RHO], ['variance', 'population']
    )]
    
    
    return hv.output(hv.Layout(curves).opts(width=900, **options))

## Explicit

In [ ]:
N = 100
a, b = [0, 10]
dt, T = 0.002, 5

theta_args = {'theta': 4.5, 'speed': 0.001, 'std_noise': 0.05}
Z, Us, THETA = solve(solve_explicit, dt, T, moving_theta=False, noise_theta=False, **theta_args)

plot_2D(Us, dt, T, dynamic=True, nb_images=50)
plot_params(Z, Us, THETA)

## Implicit

In [ ]:
N = 100
a, b = [0, 10]
dt, T = 0.02, 30

theta_args = {'theta': 4.5, 'speed': 0.001, 'std_noise': 0.5}
Z, Us, THETA = solve(solve_implicit, dt, T, moving_theta=False, noise_theta=True, **theta_args)

plot_2D(Us, dt, T, dynamic=False, nb_images=10)
plot_params(Z, Us, THETA)

## Splitting

In [ ]:
N = 150
a, b = [-5, 10]
dt, T = 0.01, 20

theta_args = {'theta': 4.5, 'speed': -0.002, 'std_noise': 0.2}
Z, Us, THETA = solve(solve_splitting, dt, T, moving_theta=True, noise_theta=False, **theta_args)

plot_2D(Us, dt, T)
plot_params(Z, Us, THETA)